<a href="https://colab.research.google.com/github/tc19r/FusedNet/blob/main/unit_tes_of_fusednet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
tf.keras.backend.set_floatx('float64')
from tensorflow.python.keras.layers import concatenate
from sklearn.mixture import GaussianMixture
from memory_profiler import profile
import numpy as np

@profile
def run():
    #Load {data -> x_train, y_train}
    X=np.empty([0,34])
    rootpath='./'
    for root, dirs, files in os.walk(rootpath):
         for filename in [fi for fi in files if fi.endswith(".pf")]:
             pf=np.fromfile(filename)
             pf=pf.reshape(int(len(pf)/34),34)
             np.random.shuffle(pf)
             X=np.vstack([X,pf])
             print(pf.shape)
             print(filename)
    trainset=np.float64(X)

    num_classes=2
    dim_fib=30
    from sklearn.mixture import GaussianMixture
    gm = GaussianMixture(n_components=num_classes, random_state=0,warm_start=True,max_iter=10)
    dataset=trainset[:,0:8]
    #Run
    #Random sampling, Acts as shufflling when shrink = 1  
    shrink=1
    indexes=np.random.choice(dataset.shape[0], int(np.floor(dataset.shape[0]/shrink)), replace=False)
    x_train=trainset[indexes,:]
    y_train=trainset[indexes,0:dim_fib]
    
    #Hyper parameters 
    learningrate,lambda1,lambda2=0.0003,1.5e-2,3e-4
    epochs = 10
    rootpath="./"
    num_classes=2
    Centers_Lengths_dimension=4
    fibs_input_dim=30
    dim_fib=30
    dim_ctnl=4
    recon_dim=fibs_input_dim
    inputs_center_and_length = tf.keras.Input(shape=(Centers_Lengths_dimension,))
    inputs_fibs_on_ball = tf.keras.Input(shape=(fibs_input_dim,))
    encoding_dim = 4
    
    def getmodel():
        encoder=keras.layers.Dense(16, activation='tanh')(inputs_fibs_on_ball)
        encoder=keras.layers.Dense(8, activation='tanh')(encoder)
        encoder=keras.layers.Dense(4, activation='tanh')(encoder)
        encoder=keras.layers.Dense(encoding_dim, activation='tanh',name="feature")(encoder)
        encoder=keras.Model(inputs=inputs_fibs_on_ball, outputs=encoder)


        decoder=keras.layers.Dense(4,activation='tanh')(encoder.outputs[0])
        decoder=keras.layers.Dense(8, activation='tanh')(decoder)
        decoder=keras.layers.Dense(16,activation='tanh')(decoder)
        decoder=keras.layers.Dense(recon_dim,activation='linear')(decoder)
        decoder=keras.Model(inputs=encoder.inputs, outputs=decoder)

        combined = concatenate([decoder.output, encoder.output,inputs_center_and_length,],axis=1)
        fusednet = keras.Model(inputs=[inputs_fibs_on_ball,inputs_center_and_length],outputs=combined)
        feature_extractor = keras.Model(
           inputs=inputs_fibs_on_ball,
           outputs=fusednet.get_layer(name="feature").output,
        )
        return fusednet,feature_extractor,
    fusednet,feature_extractor = getmodel()

    
    import time
    model=fusednet
    optimizer = tf.keras.optimizers.Adam(learning_rate=learningrate)
    #would the trainning metric made the update faster? scope?
    for epoch in range(epochs):
      print("\nStart of epoch %d" % (epoch,))
      start_time = time.time()

    # Iterate over the batches of the dataset.
      with tf.GradientTape() as tape:
          y_pred = model([x_train[:,0:dim_fib],x_train[:,dim_fib:dim_fib+dim_ctnl]], training=True)
          z = y_pred[:,recon_dim:]
          gm.fit(z)
          gamma = gm.predict_proba(z)
          gamma_sum = tf.reduce_sum(gamma, axis=0)
          phi = tf.reduce_mean(gamma, axis=0)
          mu = gm.means_
          Sigma = gm.covariances_
          n_features = z.shape[1]
          min_vals = tf.linalg.diag(tf.ones(n_features, dtype=tf.float64)) * 1e-6
          z_centered = z[:, None, :] - mu[None, :, :]
          L = tf.linalg.cholesky(Sigma + min_vals[None, :, :])
          log_det_sigma = 2.0 * tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)), axis=1)
          v = tf.linalg.triangular_solve(L, tf.transpose(z_centered, [1, 2, 0]))
          d = z_centered.shape[1]
          logits = tf.dtypes.cast(tf.math.log(phi[:, None]), dtype=tf.float64) - 0.5 * (
                        tf.dtypes.cast(tf.reduce_sum(tf.square(v), axis=1), dtype=tf.float64) + tf.dtypes.cast(
                    d * tf.math.log(2.0 * np.pi), dtype=tf.float64) + tf.dtypes.cast(log_det_sigma[:, None],dtype=tf.float64))
          energies = - tf.reduce_logsumexp(logits, axis=0)
          enerloss = tf.reduce_mean(energies)
          input = y_train[:, 0:recon_dim]
          recon = y_pred[:,0:recon_dim]
          reconloss = tf.math.reduce_sum(tf.reduce_mean(tf.square(recon - input), axis=-1))
          loss_value=lambda1*enerloss + reconloss
          grads = tape.gradient(loss_value, model.trainable_weights)
          optimizer.apply_gradients(zip(grads, model.trainable_weights))
      print("Loss:",float(loss_value))
      print("Time taken: %.2fs" % (time.time() - start_time))
    #save model
    fusednet.save("model.h5")
    #save gmm
    import pickle
    with open('gm.pkl','wb') as f:
      pickle.dump(gm,f)

if __name__ == '__main__':
    run()



In [ ]:
#Just get the visualization down too cause bored.

#6/9/2022: 
#environmental
import nibabel
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

#Load data
import os
rootpath='./'
X=np.empty([0,34])
for root, dirs, files in os.walk(rootpath):
     for filename in [fi for fi in files if fi.endswith(".pf")]:
        pf=np.fromfile(filename)
        pf=pf.reshape(int(len(pf)/34),34)
        np.random.shuffle(pf)
        X=np.vstack([X,pf])
        print(pf.shape)
trainset=np.float64(X)
x_train=trainset

#parameters

#Load Model
from tensorflow import keras
loaded = keras.models.load_model("model.h5")

#Load gmm:
import pickle
with open('gm.pkl', 'rb') as f:
    gm = pickle.load(f)

#Get features:
feature_extractor = keras.Model(
inputs=loaded.inputs,
outputs=loaded.get_layer(name="feature").output,)
fet=feature_extractor.predict([x_train[:,0:30],x_train[:,30:34]]) #?
fullfet=np.hstack([fet,x_train[:,30:34]])

##Create bundles:
#retrieve labels from gmm
#gm = GaussianMixture(n_components, random_state=123).fit(fullfet)
labels=gm.predict(fullfet)
activeclasses=np.unique(labels)
#Acquire classes membership
index=[]
for cl in activeclasses:
    index.append(tf.where(labels==cl).numpy())

#Old bundleing routines
#preload .trk first
from dipy.io.streamline import load_trk
# trkpath='./
# for root, dirs, files in os.walk(rootpath):
#      for filename in [fi for fi in files if fi.endswith(".pf")]:
#         trkname=os.path.join(trkpath,filename[0:6])
#         trkname_list.append(trkname)
ct1=load_trk('CC_3.trk', "same", bbox_valid_check=False)
streamlines=ct1.streamlines

#create new bundles
o_c_i=[]
for i in range(len(activeclasses)):
    #o_c_i.append(index[i][np.where((index[i])<X.shape[0])[0]])
    o_c_i.append(index[i])

from dipy.io.streamline import load_tractogram, save_tractogram
for j in range(len(activeclasses)):
    nasq=streamlines[o_c_i[j][:,0]]
    newSft=ct1
    newSft.streamlines=nasq
    save_tractogram(newSft, './Pops/cluster_%d.trk' % j, bbox_valid_check=False)





Losses goes down pretty slowly, 70-~50.